In [ ]:
import datetime

def hora_atual():
    data_utc = datetime.datetime.now(datetime.timezone.utc)
    fuso_horario_sp = datetime.timezone(datetime.timedelta(hours=-3))
    data_sp = data_utc.astimezone(fuso_horario_sp)
    data_hora_formatada = data_sp.strftime('%d/%m/%Y %H:%M:%S')
    return data_hora_formatada

class ItemEstoque:
    def __init__(self, id_item, nome, quantidade, custo_unitario):
        self.id_item = id_item
        self.nome = nome
        self.quantidade = quantidade
        self.custo_unitario = custo_unitario

class Estoque:
    def __init__(self):
        self.itens = {}

    def adicionar_item(self, id_item, nome, quantidade, custo_unitario):
        if id_item in self.itens:
            self.itens[id_item].quantidade += quantidade
        else:
            self.itens[id_item] = ItemEstoque(id_item, nome, quantidade, custo_unitario)

    def calcular_custo_total(self):
        custo_total = sum(item.quantidade * item.custo_unitario for item in self.itens.values())
        return custo_total

    def gerar_relatorio(self):
        relatorio = []
        for item in self.itens.values():
            relatorio.append({
                "ID": item.id_item,
                "Nome": item.nome,
                "Quantidade": item.quantidade,
                "Custo Unitário": item.custo_unitario
            })

        def ordem_permitida():
            while True:
                try:
                    ordenacao = input("Digite a ordem de classificação (ID, Nome, Quantidade ou Custo Unitário): ").upper()
                    if ordenacao in ["ID", "NOME", "QUANTIDADE", "CUSTO UNITÁRIO"]:
                        return ordenacao
                    else:
                        print("Opção inválida. Tente novamente.")
                except ValueError:
                    print("Entrada inválida. Por favor, digite um dos campos válidos.")

        ordenacao_escolhida = ordem_permitida()
        if ordenacao_escolhida == "ID":
            relatorio.sort(key=lambda item: item["ID"])
        elif ordenacao_escolhida == "NOME":
            relatorio.sort(key=lambda item: item["Nome"])
        elif ordenacao_escolhida == "QUANTIDADE":
            relatorio.sort(key=lambda item: item["Quantidade"])
        else:
            relatorio.sort(key=lambda item: item["Custo Unitário"])
        relatorio_gerado = "\n".join(str(item) for item in relatorio)
        return relatorio_gerado

def remover_item(estoque):
    print("\nEstado Atual do Estoque:")
    print("\n", estoque.gerar_relatorio(), "\n")

    while True:
        item_input = input("Digite o ID ou o nome do item a ser removido ou digite 0 para voltar: ")

        if item_input == '0':
            print("Voltando ao Menu Principal...")
            return

        item_encontrado = False
        item_id = None

        if item_input.isdigit():
            item_id = int(item_input)
            if item_id in estoque.itens:
                item_encontrado = True
        else:
            for id_item, item in estoque.itens.items():
                if item.nome.lower() == item_input.lower():
                    item_id = id_item
                    item_encontrado = True
                    break

        if item_encontrado:
            nome_item = estoque.itens[item_id].nome
            del estoque.itens[item_id]
            print(f"Item '{nome_item}' removido do estoque com sucesso.")
            return
        else:
            print(f"Erro: Item '{item_input}' não encontrado.")



def analisar_sustentabilidade(desperdicio, vendas):
    desperdicio_total = sum(desperdicio.values())
    vendas_total = sum(vendas.values())
    total_retirado = desperdicio_total + vendas_total
    percentual_desperdicio = (desperdicio_total / total_retirado) * 100 if total_retirado > 0 else 0
    return desperdicio_total, percentual_desperdicio

def descartar_item(estoque, analisar_sustentabilidade, desperdicio):
    print("\nEstado Atual do Estoque:")
    print("\n", estoque.gerar_relatorio(), "\n")

    while True:
        item_input = input("Digite o ID ou o nome do item a ser removido ou digite 0 para voltar: ")

        if item_input == '0':
            print("Voltando ao Menu Principal...")
            return desperdicio

        quantidade = input_int("Digite a quantidade a ser descartada: ")

        item_encontrado = False
        item_id = None

        if item_input.isdigit():
            item_id = int(item_input)
            if item_id in estoque.itens and estoque.itens[item_id].quantidade >= quantidade:
                item_encontrado = True
        else:
            for id_item, item in estoque.itens.items():
                if item.nome.lower() == item_input.lower() and item.quantidade >= quantidade:
                    item_id = id_item
                    item_encontrado = True
                    break

        if item_encontrado:
            estoque.itens[item_id].quantidade -= quantidade
            desperdicio[item_id] = desperdicio.get(item_id, 0) + quantidade
            print(f"Descarte realizado com sucesso! {quantidade} unidades do item '{estoque.itens[item_id].nome}' descartadas.")
            return desperdicio
        else:
            if item_id is None:
                print(f"Erro: Item '{item_input}' não encontrado.")
            else:
                print(f"Erro: Item '{item_input}' não possui {quantidade} unidades disponíveis.")


def prever_reposicao(historico_vendas_reposicao, estoque):
    print("\nEstado Atual do Estoque:")
    print("\n", estoque.gerar_relatorio(), "\n")

    item_input = input("Digite o ID ou o nome do item a ser calculada a reposição: ")
    dias_previsao = input_int("Digite o número de dias para previsão: ")

    item_id = None
    item_encontrado = False

    if item_input.isdigit():
        item_id = int(item_input)
        if item_id in estoque.itens:
            item_encontrado = True
    else:
        for id_item, item in estoque.itens.items():
            if item.nome.lower() == item_input.lower():
                item_id = id_item
                item_encontrado = True
                break

    if item_encontrado:
        total_vendas_item = 0
        dias_com_vendas = 0

        for (dia, item_id_venda), quantidade_vendida in historico_vendas_reposicao:
            if item_id_venda == item_id:
                total_vendas_item += quantidade_vendida
                dias_com_vendas += 1

        if dias_com_vendas > 0:
            media_vendas_por_dia = total_vendas_item / dias_com_vendas
            quantidade_reposicao = media_vendas_por_dia * dias_previsao - estoque.itens[item_id].quantidade
        else:
            media_vendas_por_dia = 0
            quantidade_reposicao = 0

        if quantidade_reposicao < 0:
            quantidade_reposicao = 0

        print(f"Previsão de reposição para {dias_previsao} dias do Item {item_id}: {quantidade_reposicao:.2f} unidades")
    else:
        if item_id is None:
            print(f"Erro: Item '{item_input}' não encontrado.")

def input_int(mensagemint):
    while True:
        try:
            return int(input(mensagemint))
        except ValueError:
            print("Por favor, insira um número inteiro válido.")

def input_float(mensagemfloat):
    while True:
        try:
            return float(input(mensagemfloat))
        except ValueError:
            print("Por favor, insira um número válido.")

def input_string(mensagemstring):
    while True:
        try:
            return str(input(mensagemstring))
        except ValueError:
            print("Por favor, insira uma informação válida.")

def vendas_do_dia(estoque, historico_vendas, historico_vendas_reposicao, vendas):
    while True:
        print("\nVendas do Dia:")
        print("1. Cadastrar Nova Venda")
        print("2. Visualizar Vendas Realizadas")
        print("3. Encerrar Dia e Calcular Lucros")
        print("4. Voltar ao Menu Principal")

        escolha_vendas = input_int("Escolha uma opção: ")

        if escolha_vendas == 1:
            print("\nEstado Atual do Estoque:")
            print("\n", estoque.gerar_relatorio(), "\n")

            item_input = input("Digite o ID ou o nome do item vendido: ")
            quantidade_vendida = input_int("Digite a quantidade vendida: ")
            if item_input.isdigit():
                item_input_normalizado = str(int(item_input))
            else:
                item_input_normalizado = item_input
            item_encontrado = False
            for item_id, item in estoque.itens.items():
                if str(item_id) == item_input_normalizado or item.nome.lower() == item_input.lower():
                    item_id = item_id
                    item_encontrado = True
                    break

            if item_encontrado and estoque.itens[item_id].quantidade >= quantidade_vendida:
                estoque.itens[item_id].quantidade -= quantidade_vendida
                historico_vendas.append((hora_atual(), item_id, quantidade_vendida))
                vendas[item_id] = vendas.get(item_id, 0) + quantidade_vendida
                print(f"Venda realizada com sucesso! {quantidade_vendida} unidades do item {item_id} vendidas.")
            else:
                print(f"Erro: Não foi possível realizar a venda. Item '{item_input}' não encontrado ou não possui {quantidade_vendida} unidades disponíveis.")
        elif escolha_vendas == 2:
            if historico_vendas:
                print("\nHistórico de Vendas do Dia:")
                for data, item_id, quantidade in historico_vendas:
                    item_nome = estoque.itens[item_id].nome if item_id in estoque.itens else "Item Desconhecido"
                    print(f"{data}: {quantidade} unidades de {item_nome}")
            else:
                print("Ainda não há vendas registradas no histórico.")
        elif escolha_vendas == 3:
             total_vendas = 0
             lucro_total = 0
             for item_id, quantidade in estoque.itens.items():
               custo_unitario = estoque.itens[item_id].custo_unitario
               vendas_item = 0
               for data, item_id_venda, quantidade_vendida in historico_vendas:
                 if item_id_venda == item_id:
                   vendas_item += quantidade_vendida
               lucro_item = vendas_item * custo_unitario
               total_vendas += vendas_item
               lucro_total += lucro_item
               quantidades_por_dia_item = {}
             for data, item_id, quantidade_vendida in historico_vendas:
                diacorte = data[:2]
                dia = int(diacorte)
                chave = (dia, item_id)
                if chave in quantidades_por_dia_item:
                    quantidades_por_dia_item[chave] += quantidade_vendida
                else:
                    quantidades_por_dia_item[chave] = quantidade_vendida
             quantidades_por_dia_item[chave] = quantidade_vendida
             total_vendas_reposicao = {}
             for (dia, item_id), quantidade_total in quantidades_por_dia_item.items():
               total_vendas_reposicao[(dia, item_id)] = quantidade_total
             historico_vendas_reposicao.extend(total_vendas_reposicao.items())
             historico_vendas.clear()
             print(f"\nItem {estoque.itens[item_id].nome}: {vendas_item} unidades vendidas - Lucro: R$ {lucro_item:.2f}")
             print(f"\nDia concluído: {hora_atual()}")
             print(f"\nTotal de Vendas: {total_vendas} unidades")
             print(f"Lucro Total do Dia: R$ {lucro_total:.2f}")

        elif escolha_vendas == 4:
            break
        else:
            print("Escolha inválida. Por favor, escolha uma opção válida.")

def acessar_deposito(estoque, historico_vendas, historico_vendas_reposicao, desperdicio, vendas):
    while True:
        print("\nO que você gostaria de fazer?")
        print("1. Adicionar item ao estoque")
        print("2. Descartar item do estoque")
        print("3. Calcular custo total do estoque")
        print("4. Gerar relatório de estoque")
        print("5. Prever reposição para um item")
        print("6. Analisar sustentabilidade")
        print("7. Deletar item")
        print("8. Voltar ao Menu Principal")

        escolha = input_int("Escolha uma opção: ")

        if escolha == 1:
            nome_item = input_string("Digite o nome do item ou digite 'voltar' para voltar: ").upper()
            if nome_item.lower() == 'voltar':
                continue
            if nome_item in [item.nome for item in estoque.itens.values()]:
                quantidade_adicionar = input_int("Item já existente. Digite a quantidade a ser adicionada ou digite 0 para voltar: ")
                if quantidade_adicionar == 0:
                    print("Voltando...")
                    continue
                for id_item, item in estoque.itens.items():
                    if item.nome == nome_item:
                        estoque.itens[id_item].quantidade += quantidade_adicionar
                        break
                print(f"Quantidade do item '{nome_item}' atualizada com sucesso.")
            else:
                while True:
                    id_item = input_int("Digite o ID do novo item a ser cadastrado: " )
                    if id_item == 0:
                        print("Voltando...")
                        return
                    if id_item not in estoque.itens:
                        break
                    else:
                        print(f"Erro: ID '{id_item}' já existe. Por favor, insira um ID novo ou digite 0 para voltar.")
                quantidade = input_int("Digite a quantidade a ser adicionada: ")
                custo_unitario = input_float("Digite o custo unitário do item: ")
                estoque.adicionar_item(id_item, nome_item, quantidade, custo_unitario)
                print("Item adicionado ao estoque.")

        elif escolha == 2:
            desperdicio = descartar_item(estoque, analisar_sustentabilidade, desperdicio)

        elif escolha == 3:
            custo_total = estoque.calcular_custo_total()
            print(f"Custo Total do Estoque: R$ {custo_total:.2f}")

        elif escolha == 4:
            relatorio = estoque.gerar_relatorio()
            print(f"\n{relatorio}")

        elif escolha == 5:
            prever_reposicao(historico_vendas_reposicao, estoque)

        elif escolha == 6:
            desperdicio_total, percentual_desperdicio = analisar_sustentabilidade(desperdicio, vendas)
            print(f"Desperdício Total: {desperdicio_total}, Percentual de Desperdício: {percentual_desperdicio:.2f}%")
        elif escolha == 7:
            remover_item(estoque)
        elif escolha == 8:
            break
        else:
            print("Escolha inválida. Por favor, escolha uma opção válida.")

def main():
    estoque = Estoque()
    historico_vendas = []
    historico_vendas_reposicao = []
    desperdicio = {}
    vendas = {}

    while True:
        print("\nMenu Principal:")
        print("1. Acessar Depósito")
        print("2. Vendas do Dia")
        print("3. Sair")

        escolha_principal = input_int("Escolha uma opção: ")

        if escolha_principal == 1:
            acessar_deposito(estoque, historico_vendas, historico_vendas_reposicao, desperdicio, vendas)
        elif escolha_principal == 2:
            vendas_do_dia(estoque, historico_vendas, historico_vendas_reposicao, vendas)
        elif escolha_principal == 3:
            print("Até mais!")
            break
        else:
            print("Escolha inválida. Por favor, escolha uma opção válida.")

if __name__ == "__main__":
    main()



Menu Principal:
1. Acessar Depósito
2. Vendas do Dia
3. Sair
Escolha uma opção: 1

O que você gostaria de fazer?
1. Adicionar item ao estoque
2. Descartar item do estoque
3. Calcular custo total do estoque
4. Gerar relatório de estoque
5. Prever reposição para um item
6. Analisar sustentabilidade
7. Deletar item
8. Voltar ao Menu Principal
Escolha uma opção: 1
Digite o nome do item ou digite 'voltar' para voltar: coisa
Digite o ID do novo item a ser cadastrado: 2
Digite a quantidade a ser adicionada: 30
Digite o custo unitário do item: 2
Item adicionado ao estoque.

O que você gostaria de fazer?
1. Adicionar item ao estoque
2. Descartar item do estoque
3. Calcular custo total do estoque
4. Gerar relatório de estoque
5. Prever reposição para um item
6. Analisar sustentabilidade
7. Deletar item
8. Voltar ao Menu Principal
Escolha uma opção: 2

Estado Atual do Estoque:
Digite a ordem de classificação (ID, Nome, Quantidade ou Custo Unitário): id

 {'ID': 2, 'Nome': 'COISA', 'Quantidade': 